# Capítulo 03: Leitura de Tabelas Iceberg

Este notebook demonstra técnicas de leitura e análise de tabelas Iceberg.

**Objetivo:** Dominar leitura, filtros e análises em tabelas Iceberg.

**Conteúdo:**
- Leitura básica de tabelas
- Filtros e agregações
- Joins entre tabelas Iceberg
- Monitoramento e estatísticas

## 8. Resumo

Neste capítulo você aprendeu:
- ✅ Leitura básica de tabelas Iceberg
- ✅ Filtros temporais e agregações
- ✅ Joins entre múltiplas tabelas
- ✅ Monitoramento de estatísticas

**Próximo:** Capítulo 04 - Metadados e Snapshots

In [ ]:
def monitor_iceberg_table(table_path):
    """Monitora estatísticas de uma tabela Iceberg"""
    try:
        stats = con.execute(f"""
            SELECT
                count(*) as total_rows,
                count(DISTINCT date_trunc('day', event_timestamp)) as days_of_data,
                min(event_timestamp) as earliest_event,
                max(event_timestamp) as latest_event
            FROM iceberg_scan('{table_path}')
        """).fetchone()
        
        print(f"""
Estatísticas da Tabela: {table_path}
- Total de linhas: {stats[0]:,}
- Dias de dados: {stats[1]}
- Evento mais antigo: {stats[2]}
- Evento mais recente: {stats[3]}
- Timestamp da análise: {datetime.now()}
        """)
        return stats
    except Exception as e:
        print(f"❌ Erro ao monitorar: {e}")
        return None

print("✅ Função monitor_iceberg_table() criada!")

## 7. Monitoramento de Tabela

In [ ]:
# Exemplo de JOIN entre múltiplas tabelas Iceberg
safe_install_ext(con, "httpfs")

query_join = """
SELECT
    o.order_id,
    o.order_date,
    c.customer_name,
    c.region,
    p.product_name,
    p.category,
    o.quantity,
    o.total_amount
FROM iceberg_scan('s3://warehouse/orders') o
JOIN iceberg_scan('s3://warehouse/customers') c
    ON o.customer_id = c.customer_id
JOIN iceberg_scan('s3://warehouse/products') p
    ON o.product_id = p.product_id
WHERE o.order_date >= '2024-01-01'
LIMIT 100
"""

print("Exemplo de JOIN entre tabelas Iceberg:")
print(query_join)

## 6. Joins entre Tabelas Iceberg

In [ ]:
# Análise mensal de vendas
query_exemplo = """
SELECT
    date_trunc('month', order_date) as month,
    count(DISTINCT customer_id) as unique_customers,
    count(*) as total_orders,
    sum(total_amount) as revenue,
    avg(total_amount) as avg_order_value
FROM iceberg_scan('s3://analytics-bucket/sales')
WHERE order_date >= '2024-01-01'
GROUP BY month
ORDER BY month
"""

print("Exemplo de query de agregação:")
print(query_exemplo)

## 5. Agregações e Análises

In [ ]:
def read_last_n_days(table_path, n_days=7, date_column='event_timestamp'):
    """Lê últimos N dias de uma tabela Iceberg"""
    cutoff_date = (datetime.now() - timedelta(days=n_days)).strftime('%Y-%m-%d')
    
    query = f"""
        SELECT *
        FROM iceberg_scan('{table_path}')
        WHERE {date_column} >= '{cutoff_date}'
    """
    
    try:
        df = con.execute(query).df()
        print(f"✅ {len(df):,} linhas dos últimos {n_days} dias")
        return df
    except Exception as e:
        print(f"❌ Erro: {e}")
        return None


# Exemplo: recent_data = read_last_n_days('s3://bucket/events', n_days=7)
print("✅ Função read_last_n_days() criada!")

## 4. Leitura com Filtro Temporal

Função para ler apenas dados recentes.

In [ ]:
def read_iceberg(table_path, filter_date=None, limit=None):
    """
    Lê tabela Iceberg com filtros opcionais
    
    Args:
        table_path: Caminho da tabela Iceberg
        filter_date: Filtrar dados >= esta data
        limit: Limite de linhas
    """
    query = f"SELECT * FROM iceberg_scan('{table_path}')"
    
    if filter_date:
        query += f" WHERE order_date >= '{filter_date}'"
    
    if limit:
        query += f" LIMIT {limit}"
    
    return con.execute(query).df()


# Exemplo de uso
# df = read_iceberg('s3://bucket/sales', filter_date='2024-01-01', limit=1000)
print("✅ Função read_iceberg() criada!")

## 3. Função Reutilizável para Leitura

In [ ]:
# Ler tabela Iceberg e converter para DataFrame
# NOTA: Ajuste o caminho para uma tabela real
try:
    result = con.execute("""
        SELECT * 
        FROM iceberg_scan('data/iceberg/sales')
        LIMIT 10
    """).df()
    
    print("✅ Tabela lida com sucesso!")
    print(f"Primeiras linhas:")
    print(result.head())
except Exception as e:
    print(f"ℹ️  Ajuste o caminho da tabela: {e}")

## 2. Leitura Básica de Tabela Iceberg

In [ ]:
# Importar bibliotecas
import duckdb
import pandas as pd
from datetime import datetime, timedelta
import importlib.util


def safe_install_ext(con, ext_name):
    """Instala e carrega extensão com tratamento de erros"""
    try:
        con.execute(f"INSTALL {ext_name}")
        con.execute(f"LOAD {ext_name}")
        print(f"✅ {ext_name} carregado")
        return True
    except Exception as e:
        print(f"⚠️  Erro: {e}")
        return False


# Conexão e setup
con = duckdb.connect()
safe_install_ext(con, "iceberg")
print("✅ Setup completo!")

In [ ]:
# Instalar dependências
import sys
!{sys.executable} -m pip install -q duckdb pyarrow pandas

## 1. Setup Inicial